In [ ]:
# import the important libraries
import numpy as np
import pandas as pd 
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils.class_weight import compute_class_weight
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Activation,Conv2D,MaxPooling2D,Flatten,Dense,Dropout,Input,AveragePooling2D,GlobalAveragePooling2D, GlobalMaxPooling2D,BatchNormalization,Activation
from tensorflow.keras.optimizers import Adam,RMSprop,Adagrad,Nadam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import regularizers

In [ ]:
# data input
fer_directory= "/kaggle/input/emotion-detection-fer/train"
fer_directory_1= "/kaggle/input/emotion-detection-fer/test"
selected_classes = ['angry', 'fearful', 'happy', 'neutral', 'sad', 'surprised']


In [ ]:
# Data Augmentation
fer_train_aug = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,       
    rotation_range=10,          
    width_shift_range=0.01,      
    height_shift_range=0.01,    
    shear_range=0.1,            
    zoom_range=0.1,             
    horizontal_flip=True,       
    brightness_range=(0.8,1), 
    fill_mode='nearest'
)
val_data = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

# training data
fer_train_data=fer_train_aug.flow_from_directory(
    fer_directory,
    target_size=(48,48),
    classes=selected_classes,
    color_mode='grayscale',
    batch_size=64,
    shuffle=True,
    class_mode='sparse',
    subset='training'

)


# validation data
fer_val_data=val_data.flow_from_directory(
    fer_directory,
    # df_val,
    classes=selected_classes,
    target_size=(48,48),
    color_mode='grayscale',
    batch_size=64,
    shuffle=False,
    class_mode='sparse',
    subset='validation'

)


# test data
fer_test_data=ImageDataGenerator(
    rescale=1./255

).flow_from_directory(
    fer_directory_1,
    classes=selected_classes,
    target_size=(48,48),
    color_mode='grayscale',
    batch_size=64,
    shuffle=False,
    class_mode='sparse'
)

In [ ]:
images,labels=next(fer_train_data)
print(images.shape)
print(labels.shape)
print(fer_train_data.class_indices)
print(labels)


In [ ]:
class_map = dict(zip(
    fer_train_data.class_indices.values(),
    fer_train_data.class_indices.keys()
))
print(class_map)

In [ ]:
# plot the images
plt.figure(figsize=(10, 10))
for i in range(9):
    plt.subplot(3, 3, i + 1)
    plt.imshow(images[i])
    label=labels[i]
    plt.title(f"Label: {class_map[label]}")
    plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# Alex net architecture
num_classes = 6  
weight_decay = 0.0005

model = Sequential()
# specify input shape up front
model.add(Input(shape=(48, 48, 1)))

# 1st conv block
model.add(Conv2D(
    96, kernel_size=(5,5), strides=(2,2), padding='same',
    kernel_initializer="he_normal",
    kernel_regularizer=regularizers.l2(weight_decay)
))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same'))

# 2nd conv block
model.add(Conv2D(
    256, kernel_size=(5,5), padding='same',
    kernel_initializer="he_normal",
    kernel_regularizer=regularizers.l2(weight_decay)
))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same'))

# 3rd conv block
model.add(Conv2D(
    384, kernel_size=(3,3), padding='same',
    kernel_initializer="he_normal",
    kernel_regularizer=regularizers.l2(weight_decay)
))
model.add(BatchNormalization())
model.add(Activation('relu'))

# 4th conv block
model.add(Conv2D(
    384, kernel_size=(3,3), padding='same',
    kernel_initializer="he_normal",
    kernel_regularizer=regularizers.l2(weight_decay)
))
model.add(BatchNormalization())
model.add(Activation('relu'))

# 5th conv block
model.add(Conv2D(
    256, kernel_size=(3,3), padding='same',
    kernel_initializer="he_normal",
    kernel_regularizer=regularizers.l2(weight_decay)
))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same'))

# dense layers
model.add(Flatten())
model.add(Dense(
    4096,
    kernel_initializer="he_normal",
    kernel_regularizer=regularizers.l2(weight_decay)
))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(
    4096,
    kernel_initializer="he_normal",
    kernel_regularizer=regularizers.l2(weight_decay)
))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

# output
model.add(Dense(
    num_classes, activation='softmax',
    kernel_initializer="glorot_uniform",
    kernel_regularizer=regularizers.l2(weight_decay)
))

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    metrics=['accuracy']
)


model.summary()

In [ ]:
# Callbacks
callbacks=[EarlyStopping(
        monitor='val_accuracy',
        mode='max',
        patience=3,
        restore_best_weights=True,
        verbose=1
    ),
    ModelCheckpoint(
        filepath='/kaggle/working/last_cnn_alex.h5',
        save_weights_only=False,
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_accuracy',
        factor=0.2,
        patience=3,
        min_lr=1e-6,
        verbose=1
    )
]

In [ ]:
history =model.fit(
    fer_train_data,
    validation_data=fer_val_data,
    epochs=100,
    callbacks=callbacks,
)

In [ ]:
test_loss, test_acc= model.evaluate(fer_test_data)
print(f"Test Accuracy: {test_acc*100:.4f}")
print(f"Test Loss: {test_loss*100:.4f}")

In [ ]:
y_pred=model.predict(fer_test_data)
y_pred_labels=np.argmax(y_pred,axis=1)
y_true=fer_test_data.classes
class_names = list(fer_test_data.class_indices.keys())

In [ ]:
from sklearn.metrics import classification_report 
print(classification_report(y_true,y_pred_labels,target_names=class_names))


In [ ]:
from sklearn.metrics import confusion_matrix 
cm=confusion_matrix(y_true,y_pred_labels)
classes = selected_classes
sns.heatmap(cm,annot=True,xticklabels=classes, yticklabels=classes,cmap='coolwarm')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix - Multiclass')
plt.show()

In [ ]:
!pip install keras-tuner

In [ ]:
# HyperParameter Tuning by Keras Tuner
import keras_tuner as kt
def build_model(hp):
    model=Sequential()
    model.add(Input(shape=(48,48,1)))
    for i in range(6):
      if i == 0:
        filters = hp.Int(f"filters_{i}", min_value=32, max_value=128, step=32)
      elif i == 1:
        filters = hp.Int(f"filters_{i}", min_value=256, max_value=512, step=128)
      elif i == 2:


        filters = hp.Int(f"filters_{i}", min_value=256, max_value=512, step=128)
      else:
        filters = hp.Int(f"filters_{i}", min_value=256, max_value=512, step=128)

      padding=hp.Choice(f"padding_{i}",["same"])
        
      model.add(Conv2D(
            filters=filters,
            kernel_size=(3,3),
            padding=padding,
            activation='relu',
            kernel_initializer='he_normal',
            kernel_regularizer=regularizers.l2(0.0005),
            # strides=(2,2)
        ))
      model.add(BatchNormalization())
        # Add pooling layer
      if i in [0,1,2,4]:
        if hp.Choice(f"Pooling_{i}",["max","avg"])=="max" :
              model.add(MaxPooling2D(pool_size=(2,2)))
        else:
              model.add(AveragePooling2D(pool_size=(2,2)))
      model.add(Dropout(hp.Float(f"dropout{i}", 0.2, 0.5, step=0.1)))
    model.add(Flatten())
    model.add(Dense(512, activation='relu',kernel_initializer='he_normal',kernel_regularizer=regularizers.l2(0.0005)))
    model.add(Dropout(0.3))
    model.add(BatchNormalization())
    model.add(Dense(1024, activation='relu',kernel_initializer='he_normal',kernel_regularizer=regularizers.l2(0.0005)))
    model.add(Dropout(0.3))
    model.add(BatchNormalization())


    model.add(Dense(units=6,activation="softmax",kernel_initializer='glorot_uniform',kernel_regularizer=regularizers.l2(0.0005)))
    model.compile(
            optimizer=Adam(learning_rate=hp.Choice("lr",[.001,.0001,.0005])),
            loss="sparse_categorical_crossentropy",
            metrics=["accuracy"])

    return model

In [ ]:
tuner=kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    directory='/kaggle/working/keras_tuner',
    project_name='emotion_detection_1',
    overwrite=False

)

In [ ]:
callbacks=[EarlyStopping(
        monitor='val_accuracy',
        patience=3,
        restore_best_weights=True,
        verbose=1
    ),
    ModelCheckpoint(
        filepath='/kaggle/working/keras_tuner/last_cnn_1.h5',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_accuracy',
        factor=0.2,
        patience=3,
        min_lr=1e-6,
        verbose=1
    )
]


In [ ]:
tuner.search(
   fer_train_data,
   validation_data=fer_val_data,
   epochs=20,
   callbacks=callbacks,
   verbose=1)

In [ ]:
best_model = tuner.get_best_models(num_models=1)[0]
history = best_model.fit(
    fer_train_data,
    validation_data=fer_val_data,
    epochs=40,
    callbacks=callbacks
)

In [ ]:
# Save the best model
best_model.save('/kaggle/working/last_cnn_1.h5')

In [ ]:
test_loss, test_acc= best_model.evaluate(fer_test_data)
print(f"Test Accuracy: {test_acc*100:.4f}")
print(f"Test Loss: {test_loss*100:.4f}")

In [ ]:
# plot the graph between val_accuracy and training_accuracy
plt.figure(figsize=(5,5))
plt.plot(history.history['val_accuracy'],label='val_accuracy')
plt.plot(history.history['accuracy'],label='accuracy')
plt.show()

In [ ]:
y_pred=best_model.predict(fer_test_data)
y_pred_labels=np.argmax(y_pred,axis=1)
y_true=fer_test_data.classes
class_names = list(fer_test_data.class_indices.keys())
from sklearn.metrics import classification_report 
print(classification_report(y_true,y_pred_labels,target_names=class_names))

In [ ]:
from sklearn.metrics import confusion_matrix 
cm=confusion_matrix(y_true,y_pred_labels)
classes = selected_classes
sns.heatmap(cm,annot=True,xticklabels=classes, yticklabels=classes,cmap='coolwarm')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix - Multiclass')
plt.show()

Alex Net:
Test Accuracy: 41.8141
Test Loss: 1013.7605


By HyerParameterTuning
Test Accuracy: 62.2046
Test Loss: 228.7357